In [25]:
import torch
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import torch.nn.functional as F

In [26]:
class EnsembleModel(nn.Module):
    def __init__(self, num_classes):
        super(EnsembleModel, self).__init__()
        
        self.vgg = models.vgg19(pretrained=False)
        self.inception = models.inception_v3(pretrained=False, aux_logits=False)
        self.resnet = models.resnet50(pretrained=False)
        
        self.vgg = nn.Sequential(*list(self.vgg.children())[:-1])
        self.inception = nn.Sequential(*list(self.inception.children())[:-1])
        self.resnet = nn.Sequential(*list(self.resnet.children())[:-1])

        self.fc = nn.Sequential(
            nn.Linear(3 * 512, 512), 
            nn.ReLU(),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        vgg_out = self.vgg(x).view(x.size(0), -1)
        inception_out = self.inception(x).view(x.size(0), -1)
        resnet_out = self.resnet(x).view(x.size(0), -1)

        combined = torch.cat((vgg_out, inception_out, resnet_out), dim=1)
        output = self.fc(combined)
        return output

In [27]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torch.load("best_model.pth", map_location=device)
model.eval() 

C:\Users\krish\AppData\Local\Temp\ipykernel_25492\737615511.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("best_model.pth", map_location=device)


EnsembleModel(
  (vgg): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      (1

In [28]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [29]:
image_path = "Cotton_Disease/test/diseased cotton plant/dd (16)_iaip.jpg"
image = Image.open(image_path)
image = transform(image).unsqueeze(0)

In [30]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
image = image.to(device)
model = model.to(device)

In [31]:
with torch.no_grad():
    output = model(image)
    probabilities = F.softmax(output[0], dim=0)
    predicted_class = torch.argmax(probabilities).item()

class_labels = ["Diseased Cotton Leaf", "Diseased Cotton Plant", "Fresh Cotton Leaf", "Fresh Cotton Plant"]
print(f"Predicted Class: {class_labels[predicted_class]}")
print(f"Confidence Score: {probabilities[predicted_class].item():.4f}")

Predicted Class: Diseased Cotton Plant
Confidence Score: 1.0000
